In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/RetinaSmartCamera


/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera


In [ ]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn

import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.utils.data.dataset import Dataset, Subset
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import cv2
import copy

from torch._utils import _accumulate

In [ ]:
class ObjectsDataset(Dataset):
  
  def __init__(self, root_dir, frames, labels_index, labels_names, image_type):
    # image_type can be original, gray image or color-opponency. It is expected 
    # that images with these types are already available - gotten through preprocessing offline. 
    self.root_dir = root_dir
    self.labels = labels_index
    self.transforms = transforms
    self.image_type = image_type
    self.frames = frames
    self.labels_names = labels_names
    
  def __len__(self):
    return len(self.frames)
    
  def __getitem__(self, index):
    frame_path = self.frames[index]
    label_index = self.labels[index]
    plant_name = self.labels_names[label_index]
    
    image = cv2.imread(frame_path)
    image = image/ 255.0
    image=torch.from_numpy(image.astype('float32')).permute(2, 0, 1)
    
    label=torch.from_numpy(np.asarray(label_index).astype('long'))
        
    return (image, label)  

In [ ]:
import re
import os

def natural_key(string_):
    """See http://www.codinghorror.com/blog/archives/001018.html"""
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

def get_dataloader(image_type = 'original', skip = 6, batch_size = 64, rand_perm = True, shuffle = True):
    #pip install import_ipynb
    
    
    root_path = '/content/drive/My Drive/RODframes2/RODframes/'
    model_path = '/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/models/'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
  
  # Train, Validation, Test ratio = 80 : 15 : 5
    dataset_path = os.path.join(root_path, 'train')
   
    classes = ['bag','beer','book','case','coffee','cup','deodorant','eraser','hole','mouse','mug','sleep','speaker','spray','stapler','tape','tea','tissues','umbrella','watch']
    image_type_path = os.path.join(root_path, 'train/')
  
    frames = []
    labels = []
    
    master_frames = []
    master_labels = []
    
    print(image_type_path)
    min_count = 7200
    for index, obj in enumerate(classes):
      try:
        object_frames = []
      
        labels_index = []
      
        object_type_path = obj + '/light/original/'
        object_path = os.path.join(image_type_path, object_type_path)
    
        count_index = 0
      
      # create a list inside a list
        master_frames.append([])
        master_labels.append([])
      
      #for folder in os.listdir(plant_path):
      #e.g folder = 0, 1, 2                    
        #folder_path = os.path.join(plant_path, folder)
        folder_path = object_path
        for frame_index, frame in enumerate(os.listdir(folder_path)):
          if (count_index % skip == 0):
            frame_path = os.path.join(folder_path, frame)
            object_frames.append(frame_path)
            labels_index.append(index)
          count_index = count_index + 1  
          
      # Check if total number of frames per object is less than the min_count
      # The reason for this is that we need an interleaving of the frames of all the objects
        if len(object_frames) < min_count:
          min_count = len(object_frames)
      
        sorted_object_frames = sorted(object_frames, key = natural_key)  
        master_frames[index].extend(sorted_object_frames)
        master_labels[index].extend(labels_index)
        print('Number of images for class ',obj, ': ', len(sorted_object_frames))
      except OSError as err:
        print("OS error: {}".format(err))
        continue
    print(master_frames)
    for i in range(0, min_count):
      for object_index in range(0, 18):
        frames.append(master_frames[object_index][i])
        labels.append(master_labels[object_index][i])   
    
    
    # Create dataset class
    dataset = ObjectsDataset(root_path, frames, labels, classes, image_type)  
    
    train_size = int (0.8 * len(dataset))
    validation_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - validation_size
    
    if rand_perm:
      train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size, test_size])
    else:
      indices = range(len(dataset))
      lengths = [train_size, validation_size, test_size]
      
      subsets = [Subset(dataset, indices[offset - length:offset]) for offset, length in zip(_accumulate(lengths), lengths)]
      train_dataset = subsets[0]
      validation_dataset = subsets[1]
      test_dataset = subsets[2]

    dataset_dict = {'train' : train_dataset, 'validation' : validation_dataset, 'test' : test_dataset}

    dataloader = {x : torch.utils.data.DataLoader(dataset_dict[x], batch_size = batch_size, shuffle = shuffle, num_workers = 0, drop_last=True) for x in ['train', 'test', 'validation']}

    print(type(dataset[0][0]))
    dataset_sizes = {x: len(dataset_dict[x]) for x in ['train', 'test', 'validation']}
    print('Dataset size is ', dataset_sizes)

    
    return dataloader   

In [ ]:
dataset = get_dataloader(image_type = '', skip = 10, rand_perm = True, batch_size=8)

cuda:0
/content/drive/My Drive/RODframes2/RODframes/train/
Number of images for class  bag :  210
Number of images for class  beer :  217
Number of images for class  book :  212
Number of images for class  case :  197
Number of images for class  coffee :  209
Number of images for class  cup :  224
Number of images for class  deodorant :  194
Number of images for class  eraser :  212
Number of images for class  hole :  212
Number of images for class  mouse :  201
Number of images for class  mug :  210
Number of images for class  sleep :  209
Number of images for class  speaker :  205
Number of images for class  spray :  204
Number of images for class  stapler :  208
Number of images for class  tape :  200
Number of images for class  tea :  222
Number of images for class  tissues :  201
Number of images for class  umbrella :  194
Number of images for class  watch :  202
[['/content/drive/My Drive/RODframes2/RODframes/train/bag/light/original/frame13.png', '/content/drive/My Drive/RODfram

In [ ]:
class Autoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(True),
            nn.Conv2d(6,16,kernel_size=5),
            nn.ReLU(True))
    
    self.decoder = nn.Sequential(             
            nn.ConvTranspose2d(16,6,kernel_size=5),
            nn.ReLU(True),
            nn.ConvTranspose2d(6,3,kernel_size=5),
            nn.ReLU(True))
    
  def forward(self,x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x 

In [ ]:
model = Autoencoder()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

distance = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

model.to(device)
i = 0
for epoch in range(5):
  
  for data in dataset['train']:
    if i%10 == 0:
      print('Iteration {}'.format(i))
    i += 1
    img, label = data
    img, label = img.to(device), label.to(device)

    output = model(img)
    
    loss = distance(output, img)

    optimizer.zero_grad()
    loss.backward()

  print('Epoch: {}, Loss: {:.4f}'.format(epoch+1, loss.data.item()))

Iteration 0
Iteration 10
Iteration 20
Iteration 30
Iteration 40
Iteration 50
Iteration 60
Iteration 70
Iteration 80
Iteration 90
Iteration 100
Iteration 110
Iteration 120
Iteration 130
Iteration 140
Iteration 150
Iteration 160
Iteration 170
Iteration 180
Iteration 190
Iteration 200
Iteration 210
Iteration 220
Iteration 230
Iteration 240
Iteration 250
Iteration 260
Iteration 270
Iteration 280
Iteration 290
Iteration 300
Iteration 310
Iteration 320
Iteration 330
Iteration 340
Epoch: 1, Loss: 0.1968
Iteration 350
Iteration 360
Iteration 370
Iteration 380
Iteration 390
Iteration 400
Iteration 410
Iteration 420
Iteration 430
Iteration 440
Iteration 450
Iteration 460
Iteration 470
Iteration 480
Iteration 490
Iteration 500
Iteration 510
Iteration 520
Iteration 530
Iteration 540
Iteration 550
Iteration 560
Iteration 570
Iteration 580
Iteration 590
Iteration 600
Iteration 610
Iteration 620
Iteration 630
Iteration 640
Iteration 650
Iteration 660
Iteration 670
Iteration 680
Iteration 690
Epoch: 2